In [41]:
import gzip
import random
import scipy
from collections import defaultdict
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
import csv
import numpy as np
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import SVDpp
from surprise import NMF
from surprise import accuracy
from surprise.model_selection import KFold

In [42]:
from collections import defaultdict


In [43]:
dataDir = "C:/Users/云舒/21Fall/CSE258/HW3/"

In [44]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [45]:
allRatings = []
userRatings = defaultdict(list)
userIDs = []
itemIDs = []

for user,recipe,d in readCSV("trainInteractions.csv.gz"):
    userIDs.append(user)
    itemIDs.append(recipe)
    r = int(d['rating'])
    allRatings.append(r)
    userRatings[user].append(r)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])


In [46]:
data = []
recipes = []
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set)
user_item = defaultdict(list)


for user,recipe,d in readCSV("trainInteractions.csv.gz"):
    data.append(d)
    recipes.append(recipe)
    user_item[user].append(recipe)

In [47]:
allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)

for user,recipe,d in readCSV("trainInteractions.csv.gz"):
    r = int(d['rating'])
    allRatings.append(r)
    userRatings[user].append(r)
    itemRatings[recipe].append(r)

globalAverage = sum(allRatings) / len(allRatings)

userAverage = {}
itemAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
for i in itemRatings:
    itemAverage[i] = sum(itemRatings[i])/len(itemRatings[i])

In [48]:
trainsize = 400000
train = data[:trainsize]
valid = data[trainsize:]

In [49]:
user_item_train = defaultdict(set)
item_user_train = defaultdict(set)
user_item_valid = defaultdict(list)
for d in train:
    user,item = d['user_id'],d['recipe_id']
    user_item_train[item].add(user)#
    item_user_train[user].add(item)
for d in valid:
    user,item = d['user_id'],d['recipe_id']
    user_item_valid[user].append(item)

In [50]:
valid_new = defaultdict(dict)
recipes = list(set(recipes))
randomed = set()
for d in valid:
    user = d['user_id']
    recipe = d['recipe_id']
    valid_new[user][recipe]=1
    while True:
        random_int = random.randint(0,len(recipes)-1)
        random_recipe = recipes[random_int]
        if random_recipe not in user_item[user] and random_recipe not in randomed:
            valid_new[user][random_recipe] = 0
            randomed.add(random_recipe)
            break

In [51]:
pos = 0
neg = 0
cnt =0
for user in valid_new:
    for recipe in valid_new[user]:
        cnt +=1
        if valid_new[user][recipe] == 1:
            pos+=1
        if valid_new[user][recipe] == 0:
            neg +=1
cnt, pos,neg

(200000, 100000, 100000)

In [59]:

recipeCount = defaultdict(int)
totalCooked = 0

for user,recipe,_ in readCSV("trainInteractions.csv.gz"):
  recipeCount[recipe] += 1
  totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalCooked*0.645: break 

In [52]:
reader = Reader(line_format='user item timestamp rating', sep=',',skip_lines=1)
dataset = Dataset.load_from_file(dataDir + "trainInteractions.csv", reader=reader)

In [53]:
trainset, testset = train_test_split(dataset, test_size=.25)

In [54]:
algo = KNNWithMeans()
cross_validate(algo, dataset, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9818  0.9856  0.9795  0.9823  0.0025  
MAE (testset)     0.5674  0.5718  0.5684  0.5692  0.0019  
Fit time          10.01   14.09   10.62   11.57   1.80    
Test time         7.19    9.32    8.01    8.18    0.88    


{'test_rmse': array([0.98181489, 0.98558006, 0.97950759]),
 'test_mae': array([0.56737441, 0.5717802 , 0.56838712]),
 'fit_time': (10.011189937591553, 14.089692115783691, 10.622385025024414),
 'test_time': (7.191771745681763, 9.324313640594482, 8.010597705841064)}

In [39]:
res = []
for k in np.arange(0.2,1,0.1):
    cnt = 0
    correct = 0
    for user in valid_new:
        for recipe in valid_new[user]:
            cnt +=1
            r = algo.predict(user,recipe).est
            if r >= k*userAverage[user]  and valid_new[user][recipe] == 1:
                correct +=1
            if r < k*userAverage[user] and valid_new[user][recipe] == 0:
                correct +=1
    accuracy1 = correct/cnt
    res.append((k,accuracy1))

In [40]:
res

[(0.2, 0.5),
 (0.30000000000000004, 0.49621),
 (0.4000000000000001, 0.49508),
 (0.5000000000000001, 0.493805),
 (0.6000000000000001, 0.49323),
 (0.7000000000000002, 0.49233),
 (0.8000000000000003, 0.49037),
 (0.9000000000000001, 0.48721)]

In [35]:
cnt = 0
rs= {}
correct = 0
for user in valid_new:
    for recipe in valid_new[user]:
        cnt +=1
        r = algo.predict(user,recipe).est
        rs[(user,recipe)]=[]
        rs[(user,recipe)].append(r)
        if r >= userAverage[user]  and valid_new[user][recipe] == 1:
            correct +=1
        if r < userAverage[user] and valid_new[user][recipe] == 0:
                correct +=1
accuracy1 = correct/cnt

In [56]:
predictions = open("predictions_Made.txt", 'w')
for l in open("stub_Made.txt"):
    if l.startswith("user_id"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in userIDs and i in itemIDs:
        r = algo.predict(u,i).est
        if r >= userAverage[u] : 
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
        
    elif u in userIDs:
        r = userAverage[u]
        if r >= globalAverage: 
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
            
    elif i in itemIDs:
        r = itemAverage[i]
        if r >= globalAverage: 
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
        
    else:
        predictions.write(u + '-' + i + ",0\n")
        

predictions.close()